In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from xgboost import XGBClassifier # Importamos el clasificador de XGBoost
from sklearn.preprocessing import StandardScaler # Para escalar, si es necesario

In [2]:
# Cargar el archivo de datos
# Asegúrate de que la ruta sea correcta o que el archivo esté en el mismo directorio
try:
    df = pd.read_excel("ClusteringClientes.xlsx")
except FileNotFoundError:
    print("Error: Asegúrate de que 'ClusteringClientes.xlsx' esté en el directorio correcto.")
    exit()

In [3]:
df.head(2)

,EMAIL,SBol_Vend,Prom_Pagado,Sum_Pagado,%_Promo,Prom_Horas_Ant,Venta_Ant,Recencia,PAGO_EFECTIVO,PAGO_TARJETA,DESC_ADULTO,DESC_DISCAPACIDAD,DESC_ESTUDIANTE,DESC_MENOR,DESC_PETFRIENDLY,DESC_PROFESOR,DESC_PROMOCION ESPECIAL,DESC_SENECTUD,Cluster
0,00-MAZA.ESQUISTOSO@ICLOUD.COM,4,953.664978,3814.659912,6.25,2.5,0.75,190,1,0,0,0,0,0,0,0,1,0,5
1,0022ANDREAGONZALEZ@GMAIL.COM,2,504.309998,1008.619995,0.00,24.5,0.50,781,0,1,0,0,1,0,0,0,0,0,2


In [4]:
# 1. Limpieza y Definición de X y y
X = df.drop(['EMAIL', 'Cluster'], axis=1) # Excluir EMAIL y ambas columnas de cluster
y = df['Cluster']
num_clases_final = y.nunique()

In [5]:
X.head(2)

,SBol_Vend,Prom_Pagado,Sum_Pagado,%_Promo,Prom_Horas_Ant,Venta_Ant,Recencia,PAGO_EFECTIVO,PAGO_TARJETA,DESC_ADULTO,DESC_DISCAPACIDAD,DESC_ESTUDIANTE,DESC_MENOR,DESC_PETFRIENDLY,DESC_PROFESOR,DESC_PROMOCION ESPECIAL,DESC_SENECTUD
0,4,953.664978,3814.659912,6.25,2.5,0.75,190,1,0,0,0,0,0,0,0,1,0
1,2,504.309998,1008.619995,0.00,24.5,0.50,781,0,1,0,0,1,0,0,0,0,0


In [ ]:

# 2. Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Datos listos. Clases finales: {num_clases_final}. Filas de entrenamiento: {X_train.shape[0]}")


# --- 2. Entrenamiento del Modelo XGBoost Básico ---

xgb_model_baseline = XGBClassifier(
    objective='multi:softmax',
    num_class=num_clases_final,
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_estimators=100,
    random_state=42
)

print("\nIniciando entrenamiento básico de XGBoost...")
xgb_model_baseline.fit(X_train, y_train)
y_pred_baseline = xgb_model_baseline.predict(X_test)


# --- 3. Evaluación del Modelo Básico ---

print("\n" + "="*50)
print("--- Evaluación del Modelo Básico ---")

accuracy = accuracy_score(y_test, y_pred_baseline)
print(f"Precisión (Accuracy) General: {accuracy:.4f}")

print("\nInforme de Clasificación:")
print(classification_report(y_test, y_pred_baseline))


# ----------------------------------------------------------------------
## 4. Pasos Siguientes: Ajuste de Hiperparámetros (Grid Search) 🚀
# Usamos f1_weighted, ya que es la mejor métrica para multi-clase con posible desbalance.

# Definición del Grid de parámetros a probar
param_grid = {
    'n_estimators': [200, 500],
    'max_depth': [3, 6],
    'learning_rate': [0.1, 0.05],
}

print("\n" + "="*50)
print("--- Iniciando Grid Search para Optimización (Scoring: F1 Ponderado) ---")

grid_search = GridSearchCV(
    estimator=XGBClassifier(
        objective='multi:softmax', 
        num_class=num_clases_final, 
        eval_metric='mlogloss', 
        random_state=42, 
        use_label_encoder=False
    ),
    param_grid=param_grid,
    scoring='f1_weighted', # Métrica robusta para clasificación multi-clase
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
best_xgb_model = grid_search.best_estimator_

print(f"\nMejor F1-Score Ponderado encontrado: {grid_search.best_score_:.4f}")
print(f"Mejores Parámetros: {grid_search.best_params_}")

# Evaluación del Modelo Final
y_pred_final = best_xgb_model.predict(X_test)

print("\n--- Evaluación del Modelo Optimizado ---")
print(f"Precisión (Accuracy) Final en Prueba: {accuracy_score(y_test, y_pred_final):.4f}")
print(classification_report(y_test, y_pred_final))

# ----------------------------------------------------------------------
## 5. Pasos Siguientes: Importancia de las Características 💡
# Analizar qué variables son las más importantes para distinguir los clusters.

feature_importances = best_xgb_model.feature_importances_
feature_names = X.columns

# Crear un DataFrame para la visualización
df_importancia = pd.DataFrame({
    'Característica': feature_names,
    'Importancia': feature_importances
}).sort_values(by='Importancia', ascending=False)

print("\n" + "="*50)
print("--- Importancia de las Características (Top 10) ---")
print("Estas son las variables clave que definen los Clusters:")
print(df_importancia.head(10).to_string(index=False))